In [73]:
import os
from kfp.v2.dsl import component, Dataset, Input
import kfp
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from kfp.v2.google.experimental import run_as_aiplatform_custom_job
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from sklearn.linear_model import ElasticNet
from typing import Any
import time

In [79]:
def train(train_x, train_y, alpha = 0.5, l1_ratio = 0.5):
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    return lr

@component(
    packages_to_install=["sklearn", "google-cloud-aiplatform"],
    base_image="python:3.9",
)
def training_job(dataset: Input[Dataset], alpha: float = 0.5, l1_ratio: float = 0.5):
    from google.cloud import aiplatform

    print(dataset.path)
    print(dataset.uri)
    print(dataset.name)
    print(dataset.TYPE_NAME)
    path = dataset.uri.lstrip("aiplatform://v1/").split("/")
    project_id = path[1]
    location = path[3]
    dataset_id = path[5]
    _dataset = aiplatform.datasets.TabularDataset(
        dataset_name=dataset_id,
        project=project_id,
        location=location
    )
    print(_dataset)
    # aiplatform.datasets.TabularDataset()
    # # TODO get dataset
    # train_uri = os.environ
    # train_x, train_y, _, _, test_x, test_y = load_dataset
    # # TODO train model
    # model = train(train_x, train_y, alpha, l1_ratio)
    # # TODO evaluate model
    # # TODO output model
    pass

In [80]:
project_id = "fuzzylabs"
region = "europe-west4"
pipeline_root_path = "gs://wine-quality-model"

@kfp.dsl.pipeline(
    name="automl-wine-quality",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str):
    ds_op = gcc_aip.TabularDatasetCreateOp(
        project=project_id,
        display_name="wine-quality",
        gcs_source="gs://wine-quality-model/wine-quality.csv",
    )

    training_job = gcc_aip.CustomPythonPackageTrainingJobRunOp(

    )

    # training_job_instance = training_job(
    #     dataset=ds_op.outputs["dataset"]
    # )

    # dummy_args = ["--data-dir", ".", "--hptune-dict", "{}"]
    # ts = int(time.time())
    # WORKING_DIR = f"{pipeline_root_path}/{ts}"
    #
    # run_as_aiplatform_custom_job(
    #     training_job_instance,
    #     display_name="wine-quality-custom-training",
    #     worker_pool_specs=[
    #         {
    #             "containerSpec": {
    #                 "args": dummy_args,
    #                 "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
    #                 "imageUri": "gcr.io/google-samples/bw-cc-train:latest",
    #
    #             },
    #             "replicaCount": "1",
    #             "machineSpec": {
    #                 "machineType": "n1-standard-4",
    #             },
    #         }
    #     ],
    # )

    # training_job_run_op = gcc_aip.AutoMLTabularTrainingJobRunOp(
    #     project=project_id,
    #     display_name="wine-quality-automl",
    #     optimization_prediction_type="regression",
    #     dataset=ds_op.outputs["dataset"],
    #     model_display_name="wine-quality-model",
    #     target_column="quality",
    #     training_fraction_split=0.6,
    #     validation_fraction_split=0.2,
    #     test_fraction_split=0.2,
    #     budget_milli_node_hours=1000,
    # )
    #
    # endpoint_op = gcc_aip.ModelDeployOp(
    #     project=project_id, model=training_job_run_op.outputs["model"]
    # )

In [81]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='wine-quality-pipeline.json')

api_client = AIPlatformClient(project_id=project_id, region=region)

response = api_client.create_run_from_job_spec(
    'wine-quality-pipeline.json',
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id
    })